In [ ]:
!pip install -q -U google-generativeai

<h1>Importing packages and loading secret [api key] </h1>

In [ ]:
import csv, re
import google.generativeai as genai
import time
import json
from kaggle_secrets import UserSecretsClient
import pandas as pd
import numpy as np
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("API_KEY")
genai.configure(api_key=api_key)

<h1>loading FDR data</h1>

In [ ]:
df=pd.read_csv('/kaggle/input/test-fdr/FDRFactualReport_DCA09MA026_Attachment1.csv')
new_df=df.iloc[2:].copy()

<h1>Setting model generation and safety settings</h1>
<h3>model : gemini-1.5-pro-latest</h3>

In [ ]:
generation_config = {
    "temperature": 0.0,
    "top_p": 0.1,
    "top_k": 1,
    "max_output_tokens": 5000,
}
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
]
MODEL_NAME = "gemini-1.5-pro-latest"
model = genai.GenerativeModel(MODEL_NAME,
                            generation_config=generation_config,
                            safety_settings=safety_settings)

<h1>Context and values set from the FDR data for further use in prompts</h1>

In [ ]:
context = "Aircraft Type:Airbus A320, midnight was at: 01/15/09 [mm/dd/yy] 00:00:00 [HH:MM:ss]"
columns = df.columns
value_units = df.iloc[0, :].values
value_type_legend = df.iloc[1, :].values
all_results =[]

<h1>Loading transcript obtained from AWS</h1>
<h3>further convertion to csv for easy use with gemini for analysis</h3>

In [ ]:
speaker_map = {"0": "atc", "1": "pilot"}

with open("Sully.txt", "r", encoding="utf-8") as f:
    text = f.read()

pattern = r"Channel \d+ \(Speaker (\d)\):\s*(.*)"
matches = re.findall(pattern, text)

with open("output.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    # Write header row
    writer.writerow(["speaker", "conversation"])

    for speaker_code, conversation in matches:
        speaker = speaker_map.get(speaker_code, "unknown")
        writer.writerow([speaker, conversation])


transcript=pd.read_csv('/kaggle/input/transcript-for-context-sully/output.csv')

<h1>Using  gemini prompt to analyse the transcript</h1>

In [ ]:

prompt_text = f"""
    I have flight data records for {context}.
    Flight data parameters: {list(columns[1:])},
    where the units in which each of the parameters where applicable are : {list(value_units)},
    and the types of values for each parameter are : {list(value_type_legend)}.
    
    I have the transcript of the conversation between the Air traffic control operator and the pilot for the corresponding period.
    Conversation data: 
    {transcript}

    analyze the conversation data and based on that and give me the following are three seperate groups of outputs:
    1.give me the list of flight data parameters from the parameters I have given as a list to focus on for further analysis (the parameters should be from the list of flight data parameters provided for this question)
    2.what is the possible issue detected from the conversation?
    3.what aircraft components would be affected based on the analyzed issue from the conversation?
    4.what are the suggestions given by the air traffic controller?
    5.what actions were taken by the pilot?
    6.what are the next steps taken or to be taken by the pilot?
    
    
    the data is accurate and is completely reliable.
    
    I want the response to each of the questions 1 -6 as a json object.
    """
response = model.generate_content(prompt_text)
response.resolve()
if response.parts:
    analysis_results = response.parts[0].text.strip().split('\n')
    all_results.extend(analysis_results)
else:
    all_results.append("No analysis provided.")

In [ ]:
transcript_result=all_results.copy()

<h1>using the ouput from the previous prompt to select important columns</h1>
<h3>reduces analysis overhead</h3>

In [ ]:
params=["Eng1 N1 Act","Eng1 N1 Cmd","Eng1 N2","Eng1 Fuel Flow","Eng1 EGT","Eng2 N1 Act","Eng2 N1 Cmd","Eng2 N2","Eng2 Fuel Flow","Eng2 EGT","Altitude Radio 1","Altitude Radio 2","Airspeed Ind","Pitch","Roll ","Heading",
 "Flap Lever Pos","Flap Pos","Slat Pos","Spoiler 1 Status","Spoiler 2 Status","Spoiler 3 Status","Spoiler 4 Status","Spoiler 5 Status","Event","Latitude","Longitude"]

<h1>Using gemini to analyze fdr based insights provided from transcript analysis</h1>

In [ ]:
def conv_est(x):
    return int(float(x))
new_df['EST_full_second']=new_df['EST'].apply(conv_est)
fin_df=new_df.loc[new_df[params].dropna(how='all').index][[new_df.columns[0]]+[new_df.columns[-1]]+params]

In [ ]:
#converting daaframe to row wise dictionaries to handle nana values for millisecond changes
mast_list=[]
fin_df1=fin_df.drop(['EST_full_second'],axis=1).copy()
for i, j in fin_df1.loc[fin_df1.index].iterrows():
    mast_list+=[j.dropna().to_dict()]

In [ ]:
all_results=[]
for i in range(0,fin_df1.shape[0], fin_df1.shape[0]//20):
    prompt_text= f"""
        I have flight data records for {context}.
        The detected issue with the flight is : Dual engine thrust loss/failure
        Flight data parameters: {list(columns)},
        where the units in which each of the parameters where applicable are : {list(value_units)},
        and the types of values for each parameter are : {list(value_type_legend)}.
    
        I have a list of flight data information which is stored as a dictionary whose key consists of 'EST' and some other keys from flight the list of parameters provided in Flightdata parameters and the values are the corresponding values for each key.
        flight data: 
        {mast_list[i:min(i + fin_df1.shape[0]//20, fin_df1.shape[0])]}
    
        Analyze the flight data and give me the values from the EST column for which anamolies are detected in the flight parametes or where the flight data parametes are suspicious or abnormal.
        Furthermore, include the name of the parameter or parameters  that shopws the most risk and their corresponding values. 
             
        
        The data is accurate and is completely reliable.
        
        I want the response as a list of json objects for each risky EST column value with the following keys:
        1. 'EST'
        2. 'risk_parameters'
        3. 'risk_parameter_values'
    
        the values from 1-3 should be the only output
        """
    
    
    
        
    response = model.generate_content(prompt_text)
    response.resolve()
    if response.parts:
        analysis_results = response.parts[0].text.strip().split('\n')
        all_results.extend(analysis_results)
    else:
        all_results.append("No analysis provided.")
    time.sleep(65) #to respect api rate limits

In [ ]:
flight_data_analysis=all_results.copy()

<h1>Conversion of json object into dataframe</h1>

In [ ]:
#from flight_data_analysis
json_data1 = """[
  {
    "EST": "54404.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["1.500", "0"]
  },
  {
    "EST": "54404.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["1.500", "0"]
  },
  {
    "EST": "54405.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["3.875", "0"]
  },
  {
    "EST": "54405.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["3.875", "0"]
  },
  {
    "EST": "54406.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["6.125", "0"]
  },
  {
    "EST": "54406.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["6.125", "0"]
  },
  {
    "EST": "54407.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["10.250", "0"]
  },
  {
    "EST": "54407.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["10.250", "0"]
  },
  {
    "EST": "54424.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["59.500", "688"]
  },
  {
    "EST": "54424.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["59.500", "688"]
  }
]"""

json_data2 = """[
  {
    "EST": 54462.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54464.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54467.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54468.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54469.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54470.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54471.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54472.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54473.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54474.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54475.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54476.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54477.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54478.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54479.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54480.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54481.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54482.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54483.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54484.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54485.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54486.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54487.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54488.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54489.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54490.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54491.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54492.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54493.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54494.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54495.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54496.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54497.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54498.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54499.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54500.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54501.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54502.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54503.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54504.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54505.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54506.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54507.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54508.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54509.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54510.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54511.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54512.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54513.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54514.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54515.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54516.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54517.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54518.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54519.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54520.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54521.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54522.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54523.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54524.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54525.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54526.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54527.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54528.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54529.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54530.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54531.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54532.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54533.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54534.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  }
]"""


def process_json_data(json_str):
    data = json.loads(json_str)
    rows = []
    for entry in data:
        row = {}
        est_val = entry.get("EST")
        if isinstance(est_val, str):
            row["EST"] = int(float(est_val))
        else:
            row["EST"] = int(est_val)
        
        for param, value in zip(entry.get("risk_parameters", []), entry.get("risk_parameter_values", [])):
            row[param] = value
        rows.append(row)
    return rows


rows1 = process_json_data(json_data1)
rows2 = process_json_data(json_data2)


all_rows = rows1 + rows2

risk_columns = set()
for row in all_rows:
    risk_columns.update(row.keys())
risk_columns.discard("EST")
risk_columns = sorted(list(risk_columns)) 

final_columns = ["EST"] + risk_columns

js_df = pd.DataFrame(all_rows, columns=final_columns)

csv_filename = "flight_data_converted.csv"
js_df.to_csv(csv_filename, index=False)


<h1>Cleaning and creation of risk score </h1>
<h3>merging data with required data for hardware </h3>

In [ ]:
est=js_df[['EST']]
def error_fill(x):
    return 0
new_df['risk']=new_df['EST'].apply(error_fill)
new_df['risk']=new_df.loc[new_df['EST_full_second'].isin(est[est.columns[0]].values)]['risk'].replace(0,1)
proc_df=new_df.iloc[[i for i in range(1526, 14678)]].drop_duplicates(subset='EST_full_second').replace(np.nan,0)[['EST_full_second','risk']].copy() #based on risk data index wrtmain FDR data 
err_dat=js_df.copy()
err_dat1=err_dat.drop_duplicates(subset='EST')

In [ ]:
def err_proc(x):
    if(int(x) in err_dat1['EST'].values):
        return [list(j.dropna().to_dict().keys())[1:] for i,j in err_dat1.loc[err_dat1['EST']==int(x)].iterrows()][0]
    else:
        return np.nan
proc_df['error']=proc_df['EST_full_second'].apply(err_proc)

In [ ]:
addl_cols=['Airspeed Ind','Altitude Radio 1', 'Heading', 'Latitude','Longitude']
dl=[]
for x in proc_df['EST_full_second'].values:
    d={"EST":x}
    for col in addl_cols:
        #[x for x in lst if pd.notna(x)]
        val=list(new_df.loc[new_df['EST_full_second']==x][col].unique())
        val_cl=[temp for temp in val if pd.notna(temp)]
        if(len(val_cl)==0):
            val_cl=[np.nan]
        
        d[col]=val_cl[-1]
    dl.append(d)

In [ ]:
proc_df1=pd.DataFrame(dl)
merged_df = pd.merge(proc_df1, proc_df, left_on='EST', right_on='EST_full_second', how='inner')
fin_df1=merged_df.drop(['EST_full_second'],axis=1)
fin_df1.to_csv('new_error.csv',index=False)